In [4]:
!pip install textract-trp
!pip install preprocess

In [5]:
!pip install utils

In [6]:
from colorama import Fore

In [7]:
import boto3
import time
import sagemaker
import os 
import trp
# from util.preprocess import *
import pandas as pd
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/commercial_notes'

In [ ]:
# fileName =  'sample_report_1.pdf'
# fileUploadPath = os.path.join('./data', fileName)
# textractObjectName = os.path.join(prefix, 'data', fileName)

# # Upload medical report file
# boto3.Session().resource('s3').Bucket(bucket).Object(textractObjectName).upload_file(fileUploadPath)

In [8]:
textract = boto3.client('textract')
response = textract.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': 'sg-doc-extract-demo',
            'Name': 'Commercial Lease Insurance Company.pdf'
        }},
    FeatureTypes=[
        'TABLES',
    ]
    )

textractJobId = response["JobId"]
print('job id is: ',textractJobId)

job id is:  17f1b1df6b66311fa2c2b90a369e9a29688bd5d0be526219ace6dc74d3746da7


In [9]:
%%time
time.sleep(5)
response = textract.get_document_analysis(JobId=textractJobId)
status = response["JobStatus"]

while(status == "IN_PROGRESS"):
    time.sleep(5)
    response = textract.get_document_analysis(JobId=textractJobId)
    status = response["JobStatus"]
    print("Textract Job status: {}".format(status))

Textract Job status: IN_PROGRESS
Textract Job status: IN_PROGRESS
Textract Job status: SUCCEEDED
CPU times: user 75.2 ms, sys: 209 µs, total: 75.4 ms
Wall time: 20.3 s


In [10]:
%%time
pages = []

time.sleep(5)

response = textract.get_document_analysis(JobId=textractJobId)

pages.append(response)

nextToken = None
if('NextToken' in response):
    nextToken = response['NextToken']

while(nextToken):
    time.sleep(5)

    response = textract.get_document_analysis(JobId=textractJobId, NextToken=nextToken)

    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
CPU times: user 324 ms, sys: 17 ms, total: 341 ms
Wall time: 36.7 s


In [11]:
doc = trp.Document(pages)
print("Total length of document is",len(doc.pages))
idx=1
for page in doc.pages:
    print(f"Results from page {idx}: \n", page.text)
    idx=idx+1

Total length of document is 4
Results from page 1: 
 STATE OF ALABAMA
COUNTY OF MOBILE
COMMERCIAL LEASE
THIS LEASE is made this the
11th
day of
April
2024
by and between
CLEOPATRA BENNETTE
of
Doroville, Georgia
("Lessor") and
Linda Cowell
of
Atlanta, GA
("Lessee.") The Agent for Lessor is
Amber Cryer
("Agent")
WITNESSETH: That the Lessor does hereby demise and let unto the Lessee the following described real property in
Monroe - GA
County, Alabama, to-wit, together with improvements thereon ("Premises"):
1. DESCRIPTION
726 Dunblane Drive Macon, GA 31210
LL 195 LD 13 LO 4 GLEN MERRY PH II
2. TERM
for continuous use and occupancy by the Lessee as Insurance Company
and for no other different use or purpose, for and during the term of
beginning on the
29th
day
of
May
2024
and
ending
on
the
29th
day of
May 2027
IN CONSIDERATION WHEREOF, the Lessee promises and agrees to pay the Lessor, or Agent at the Office of
Agent, as fixed minimum rent for the Premises for the whole term hereunder, the 

Extract  information with Amazon Comprehend Medical

In [39]:
maxLength=20000
comprehendResponse = []
comprehend_client = boto3.client(service_name='comprehend', region_name='us-east-1')

for page in doc.pages:
    pageText = page.text
    
    for i in range(0, len(pageText), maxLength):
        response = comprehend_client.detect_entities(
            Text=pageText[0+i:maxLength+i],
            LanguageCode='en')
        comprehendResponse.append(response)

In [40]:
#Utility
## Function to extract a single record
def extract_entities(json_file):
    
    ## initialize the list variables for medical_conditions, scores and traits
    entities_data=[]
    scores=[]
    entity_type=[]
    
    for row in json_file['Entities']:
        # 
        if row['Type'] == "PERSON":
            entities_data.append(row['Text'])#  += row['Text'] + ' '
            scores.append(row['Score'])
            entity_type.append("Person")
        if row['Type'] == "LOCATION":
            entities_data.append(row['Text'])#  += row['Text'] + ' '
            scores.append(row['Score'])
            entity_type.append("Location")
        if row['Type'] == "DATE":
            entities_data.append(row['Text'])#  += row['Text'] + ' '
            scores.append(row['Score'])
            entity_type.append("Date")
        if row['Type'] == "QUANTITY":
            entities_data.append(row['Text'])#  += row['Text'] + ' '
            scores.append(row['Score'])
            entity_type.append("Quantity")
        if row['Type'] == "OTHER":
            entities_data.append(row['Text'])#  += row['Text'] + ' '
            scores.append(row['Score'])
            entity_type.append("Other")
            
            
#             trait='NaN'
#             if row['Traits']:
#                 #print(row['Traits'],row['Text'] )
#                 trait=row['Traits'][0]['Name']
                
#             traits.append(trait)
        df_mc = pd.DataFrame({'Data': pd.Series(entities_data), 'Score':pd.Series(scores),'Type':pd.Series(entity_type)})
    return df_mc

In [41]:
df_cm=extract_entities(comprehendResponse[0])
df_cm['ID']=1
df_cm.head(100)

,Data,Score,Type,ID
0,STATE OF ALABAMA\nCOUNTY,0.566362,Location,1
1,11th\nday of\nApril\n2024,0.991839,Date,1
2,BENNETTE,0.894958,Person,1
3,"Doroville, Georgia",0.991946,Location,1
4,Linda Cowell,0.998988,Person,1
5,"Atlanta, GA",0.992871,Location,1
6,Amber Cryer,0.996822,Person,1
7,"Monroe - GA\nCounty, Alabama",0.962295,Location,1
8,726,0.731012,Location,1
9,"Dunblane Drive Macon, GA 31210\nLL",0.822919,Location,1
